In [ ]:
import pickle

In [ ]:
import vk_api
vk_session = vk_api.VkApi(token="4ee251bf489b9d88106f08b36239eb0ab39bed07ca3fb9adacf62346cb2981eed59a26f4c29a18123f053")

In [ ]:
vk_session.authorization()

In [ ]:
cached_ids = pickle.load(open("cache.p", "rb"))

In [ ]:
def get_many_of_public(group_link, vk_session):
    global cached_ids
    
    shortname = group_link[group_link.rfind("/")+1:]
    r = vk_session.method("groups.getById", {"group_ids": shortname})
    group_id = r[0]["id"]
    
    if group_id not in cached_ids.keys():
    
        """Все сообщества подписчиков сообщества"""
        users = vk_api.VkTools(vk_session).get_all("groups.getMembers", 100, {'group_id':group_id})['items']
        k = int(len(users) / 1000)
        if k:
            users = users[::k]

        data_for_clustering = []
        users_publicpages = []
        with vk_api.VkRequestsPool(vk_session) as pool:
            
            for i in range(0, len(users), 1000):
                batch = users[i:i+1000]
                data_for_clustering.append(pool.method('users.get', {
                        "user_ids": batch,
                        "fields": "sex,age,education,universities,schools,interests,music,movies,bdate,city,country"
                    }))

                for user_id in batch:
                    users_publicpages.append(pool.method('users.getSubscriptions', {
                                "user_id": user_id
                            }))

        data_for_clustering = [x for x in [x.result for x in data_for_clustering]]
        t = []
        for user in users_publicpages:
            try:
                t.append(user.result['groups']['items'])
            except:
                pass
        users_publicpages = t
        
        wall50 = vk_session.method("wall.get", {"owner_id": -group_id, "filter": "owner", "count": 100})['items'][::2]
        
        result = data_for_clustering, users_publicpages, wall50
        cached_ids[group_id] = result
        pickle.dump(cached_ids, open("cache.p", "wb"))
        return 
    else:
        return cached_ids[group_id]

In [ ]:
from time import time
t = time()
a = get_many_of_public("https://vk.com/goto_msk", vk_session)
print(time() - t)

In [ ]:
len(cached_ids[80270762][1])

In [ ]:
import pickle

In [ ]:
cached_ids = pickle.load(open("cache.p", "rb"))

In [ ]:
from sklearn.manifold import TSNE

In [ ]:
tsne = TSNE()

In [ ]:
cached_ids.keys()

In [ ]:
import pandas as pd
data = pd.read_csv("communities.csv").to_dict()

In [ ]:
indexer = pickle.load(open("community_indexer.p", "rb"))

In [ ]:
indexer = {v: k for k, v in data["link"].items()}
y = {}
for group_link in list(indexer.keys())[1:]:
    shortname = group_link[group_link.rfind("/")+1:]
    if "public" in shortname:
        shortname = shortname[6:]
    y[shortname] = indexer[group_link]
indexer = y

In [ ]:
import vk
session = vk.Session()
api = vk.API(session)

x = {}
for k, v in indexer.items():
    try:
        r = api.groups.getById(group_ids=k)
        group_id = r[0]["gid"]
        print(group_id)
        x[group_id] = v
    except:
        pass
indexer = x

In [ ]:
import numpy as np
from tqdm import tqdm_notebook
from collections import Counter

In [ ]:
X = []
Y = []

for gid in tqdm_notebook(cached_ids):
    try:
        y = data["category"][indexer[gid]]
    except:
        continue
    
    x_0 = []
    try:
        for i in range(1000):
            x = cached_ids[gid][0][0][i]
            try:
                ci = x["city"]["id"]
            except:
                ci = 0
            try:
                u = x["universities"][0]["id"]
            except:
                u = 0
            try:
                s = x["schools"][0]["id"]
            except:
                s = 0
            try:
                bd = x["bdate"]
                if len(bd.split(".")) == 3:
                    age = 2014 - int(bd.split(".")[2])
            except:
                age = 0
            x_0.append(np.array([x.get("sex", 0), ci, u, s, age], dtype="int32"))
        z_0 = []
        try:
            for j in range(50):
                wall_post = cached_ids[gid][2][j]
                has_photo, has_text, has_audio = 0, 0, 0
                if wall_post.get("text"):
                    has_text = 1
                t = wall_post.get("attachments")
                if t:
                    for a in t:
                        if a.get("type") == "audio":
                            has_audio = 1
                        if a.get("type") == "photo":
                            has_photo = 1
                z_0.append([has_text, has_photo, has_audio])
        except IndexError as e:
            print(e)
            
        t_sex = [z[0] for z in x_0]
        f1 = t_sex.count(2) / len(t_sex)
        f2 = t_sex.count(1) / len(t_sex)
        f3 = Counter([z[1] for z in x_0 if z[1]]).most_common(1)[0][0]
        f4 = Counter([z[2] for z in x_0 if z[2]]).most_common(1)[0][0]
        f5 = Counter([z[3] for z in x_0 if z[3]]).most_common(1)[0][0]
        f6_10 = [t[0] for t in Counter([item for sublist in cached_ids[gid][1] for item in sublist]).most_common(5)]
        try:
            f11 = sum([z[0] for z in z_0]) / len(z_0)
        except:
            f11 = 0
        try:
            f12 = sum([z[1] for z in z_0]) / len(z_0)
        except:
            f12 = 0
        try:
            f13 = sum([z[2] for z in z_0]) / len(z_0)
        except:
            f13 = 0
        ages = [z[4] for z in x_0 if z[4]]
        f14 = np.median(ages)
        x_1 = [f1, f2, f3, f4, f5, *f6_10, f11, f12, f13, f14]
        print(gid, x_1)
        
        X.append(np.array(x_1, dtype="float32"))
        Y.append(y)
    except IndexError as e:
        print(e)

In [ ]:
def from_cache(gid):
    x_0 = []
    try:
        for i in range(1000):
            x = cached_ids[gid][0][0][i]
            try:
                ci = x["city"]["id"]
            except:
                ci = 0
            try:
                u = x["universities"][0]["id"]
            except:
                u = 0
            try:
                s = x["schools"][0]["id"]
            except:
                s = 0
            try:
                bd = x["bdate"]
                if len(bd.split(".")) == 3:
                    age = 2014 - int(bd.split(".")[2])
            except:
                age = 0
            x_0.append(np.array([x.get("sex", 0), ci, u, s, age], dtype="int32"))
        z_0 = []
        try:
            for j in range(50):
                wall_post = cached_ids[gid][2][j]
                has_photo, has_text, has_audio = 0, 0, 0
                if wall_post.get("text"):
                    has_text = 1
                t = wall_post.get("attachments")
                if t:
                    for a in t:
                        if a.get("type") == "audio":
                            has_audio = 1
                        if a.get("type") == "photo":
                            has_photo = 1
                z_0.append([has_text, has_photo, has_audio])
        except IndexError as e:
            print(e)
            
        t_sex = [z[0] for z in x_0]
        f1 = t_sex.count(2) / len(t_sex)
        f2 = t_sex.count(1) / len(t_sex)
        f3 = Counter([z[1] for z in x_0 if z[1]]).most_common(1)[0][0]
        f4 = Counter([z[2] for z in x_0 if z[2]]).most_common(1)[0][0]
        f5 = Counter([z[3] for z in x_0 if z[3]]).most_common(1)[0][0]
#         f6_10 = [t[0] for t in Counter([item for sublist in cached_ids[gid][1] for item in sublist]).most_common(5)]
        f6_10 = [0, 0, 0, 0, 0]
        try:
            f11 = sum([z[0] for z in z_0]) / len(z_0)
        except:
            f11 = 0
        try:
            f12 = sum([z[1] for z in z_0]) / len(z_0)
        except:
            f12 = 0
        try:
            f13 = sum([z[2] for z in z_0]) / len(z_0)
        except:
            f13 = 0
        ages = [z[4] for z in x_0 if z[4]]
        f14 = np.median(ages)
        x_1 = [f1, f2, f3, f4, f5, *f6_10, f11, f12, f13, f14]
        
        return np.array(x_1, dtype="float32")
    except IndexError as e:
        print(e)